In [19]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session
import glob
import json
import os
import urllib2
from urllib2 import Request, urlopen

In [20]:
requests_cache.install_cache("cache")

def geonames_search(geonameid):
    base = 'http://api.geonames.org/childrenJSON'
    username = 'stephanielamm'
    
    params = ({"geonameId": geonameid, "username":username})
    
    response = base + "?" + "&".join("{}={}".format(a,b) for a,b in params.items())

    collreq = Request(response)
    collf = urllib2.urlopen(collreq)
    college_json = json.loads(collf.read())

    urllist = [i for i in college_json.values()]
        
    return urllist

In [28]:
def get_city_name(us_state):
    us_id = 6252001
    country = geonames_search('%d' %us_id)
    
    for i in range(0, len(country[1])):
        if country[1][i]['name'] == us_state:
            geonameid = country[1][i]['geonameId']
            states = geonames_search('%d' %geonameid)
    
    id_list = []
    #from state, extract the id of all county
    for i in range(0, len(states[1])):
        idnum = states[1][i]['geonameId']
        id_list.append(idnum)

    #extract all county info in CA
    city_list = []
    for i in range(0, len(id_list)):
        region = geonames_search('%d' %id_list[i])
        city_list.append(region)
        
    #from county, extract all city names, lat, and lng in each county
    citynames = []
    lat_list = []
    lng_list = []
    for i in range(0, len(city_list)):
        for j in range(0,len(city_list[i][1])):
            names = city_list[i][1][j]['name']
            lat = city_list[i][1][j]['lat']
            lng = city_list[i][1][j]['lng']
            citynames.append(names)
            lat_list.append(lat)
            lng_list.append(lng)

    citynames = pd.DataFrame(citynames)
    lat_list = pd.DataFrame(lat_list)
    lng_list = pd.DataFrame(lng_list)
    df = pd.concat([citynames, lat_list, lng_list], axis = 1)
    df.columns = ['City', 'Latitude', 'Longitude']

    return df

In [37]:
city_df = get_city_name("California")

In [38]:
city_df

City  Latitude   Longitude
0                        Adams Point  37.81389  -122.25528
1                            Alameda  37.76521  -122.24164
2                             Albany  37.88687  -122.29775
3                             Albrae  37.49549  -121.98941
4                          Allendale  37.78778  -122.20583
5                           Altamont  37.74382  -121.66273
6                               Asco  37.68299  -121.87273
7                            Ashland  37.69465  -122.11385
8                           Baumberg  37.62077  -122.10024
9                           Berkeley  37.87159  -122.27275
10                        Brightside   37.5991   -121.9244
11                        Brookshire  37.75909  -122.02829
12                            Canyon  37.83067  -122.16599
13                         Carpenter  37.61327  -122.04441
14                     Castro Valley   37.6941  -122.08635
15              Centerville District   37.5541  -121.99913
16                        Cherryland  37.67938   -122.1033
17                         Claremont  37.85571  -122.24423
18      Continental Mobile Home Park  37.63028  -122.06083
19                 Crocker Highlands  37.81333  -122.22583
20                           Curtner  37.46966  -121.92357
21                   Dimond District  37.80333  -122.21278
22                         Dougherty  37.71104  -121.90968
23                           Dresser  37.59243  -121.95579
24                            Dublin  37.70215  -121.93579
25                   East Pleasanton  37.67354  -121.82996
26                            Eberly  37.57244  -121.99329
27                        Emeryville  37.83132  -122.28525
28                          Fairview  37.67854   -122.0458
29                           Farwell  37.59827  -121.94524
...                              ...       ...         ...
5422                      Oak Valley   39.4985  -121.03801
5423                      Olive Hill  39.28822  -121.47024
5424                      Olivehurst  39.09545  -121.55219
5425                    Oregon House  39.35628  -121.27913
5426                          Ostrom  39.06934  -121.51163
5427            Peach Mobile Estates  39.11194  -121.56528
5428                         Pearson  39.06184  -121.57746
5429                     Plumas Lake  39.02073  -121.55802
5430                        Rackerby  39.43961  -121.34135
5431                         Ramirez  39.26739  -121.53886
5432                Rancho Loma Rica  39.30878  -121.35413
5433                   Sharon Valley  39.48211  -121.24691
5434                     Sicard Flat    39.231  -121.34607
5435                     Smartsville  39.20762  -121.30013
5436                      South Yuba  39.12545  -121.58552
5437                  Stanfield Hill  39.34239  -121.33302
5438               Strawberry Valley  39.56406   -121.1069
5439                     Sucker Flat  39.21184  -121.29718
5440                           Tambo  39.23934  -121.57719
5441     The Garden Mobile Home Park  39.12333  -121.58028
5442    Thunderbird Mobile Home Park  39.12861  -121.56083
5443                       Timbuctoo  39.21684  -121.31857
5444  Villa Seville Mobile Home Park  39.07944  -121.53222
5445             Vinboy Trailer Park      39.1  -121.54806
5446                  Waldo Junction  39.11101  -121.30912
5447                     Weeds Point  39.47739  -121.04968
5448                      West Linda  39.11833  -121.57889
5449                       Wheatland  39.00989  -121.42301
5450                        Woodleaf  39.51739  -121.19163
5451           Yuba Mobile Home Park  39.07528  -121.54556

[5452 rows x 3 columns]

In [39]:
requests_cache.install_cache("cache")

def yelp_search(term, location):
    """
    makes a search request
    
    Argument: search term, location
    
    Return: search result items as a list (for 0 items, return an empty list)
    """
    consumer_key = "uFKYftN0fu4pD6guW2cYng"
    consumer_secret = "iUYQ2Hk3-vTQVd0zOFnvloWPivA"
    token = "FZnep4FiFmwrrifuNm6M-_K661Wb3J02"
    token_secret = "CRvBo_fO-r4pE_-p2UrnN_mxBlc"
   
    urlbase = "http://api.yelp.com/v2/search"
    params = ({"term": term})
    params.update({"location": location, "sort": "1", "cll" : "latitude,longitude"})
    session = OAuth1Session(consumer_key, consumer_secret,token,token_secret)
     
    request = session.get(urlbase,params=params)
   
    #Transforms JSON API response into dictionary
    data = request.json()
    session.close()
    #data_dict = data["businesses"]
    
    return data

In [ ]:
data_list = []
for i in range(0, len(city_df['City'])):
    data = yelp_search("food", "%s" %city_df['City'][i])
    data_list.append(data)

In [ ]:
data_list

In [ ]:
data = yelp_search("food", "Sacramento")
data

In [ ]:
pd.DataFrame(data['businesses'])

In [ ]:
path = "/Users/Stephanie/Desktop/yelp_dataset_challenge_round9"
files = glob.glob(os.path.join(path,"*.json"))
files

In [ ]:
############# Copy from online(not sure if it works or not yet) ##########

#Convert Yelp Academic Dataset from JSON to CSV
import csv
def convert(line):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    data = json.loads(line)
    for k, v in data.items():
        if isinstance(v, list):
            data[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                data['%s_%s' % (k, kk)] = vv
            del data[k]
    return line


csv_filename = 'review.csv'
df = pd.DataFrame([convert(line) for line in file(files[2])])
df.to_csv(csv_filename, index=False)


In [ ]:
def readfile(files):
    # read the file into a python array
    with open(files, 'rb') as f:
        data = f.readlines()
        # remove the trailing "\n" from each line
        data = map(lambda x: x.rstrip(), data)
        data_str = "[" + ','.join(data) + "]"
        df = pd.read_json(data_str)
        
        f.close()
        return df
     
def fixdf(newlist):
    new_df = pd.concat(newlist, axis = 0)
    return new_df

yelplist = []

In [ ]:
review_data = readfile(files[2])

In [ ]:
review_data.head()